# import libs

In [1]:
import configparser
confParser = configparser.ConfigParser()

#--- parse conf. file
confParser.read('configuration.ini')
print('conf. file sections:',confParser.sections())
#
import os
import sys
sys.path.append(confParser['input files']['lib_path'])
from dscribe.descriptors import SOAP
import ase
import ase.io
import pdb
import time
import numpy as np
import pandas as pd
import traceback

#--- user modules
import LammpsPostProcess as lp
import utility as utl
import imp
imp.reload(utl)
imp.reload(lp)


conf. file sections: ['parameters', 'flags', 'MsdAnalysis', 'EnergyBarrier', 'SroAnalysis', 'input files', 'Atomic Radius']


<module 'LammpsPostProcess' from '../../HeaDef/postprocess/LammpsPostProcess.py'>

# set parameters

In [2]:
#--- set dynamic parameters
temp = confParser['parameters']['temperature']

#--- edit list of input files
confParser.set('input files','dump file',''.join([
               'allconf ', #0
              ]))
confParser.set('input files','diffusion file',''.join([
               'Diffusion.dat ', #0
              ]))


In [3]:
# atoms = ase.io.read("equilibrated.dat",format="lammps-data",style='atomic')

# species = 'Co Ni Cr Fe Mn'.split()
# r_cut = 6.0
# n_max = 8
# l_max = 6

# # Setting up the SOAP descriptor
# soap = SOAP(
#     species=species,
#     periodic=True,
#     r_cut=r_cut,
#     n_max=n_max,
#     l_max=l_max,
# )


# # set atomic numbers for each species
# numbers = list(map(lambda x:soap.index_to_atomic_number[x-1],atoms.get_atomic_numbers()))
# atoms.set_atomic_numbers(numbers)
# #assert soap.check_atomic_numbers(atoms.get_atomic_numbers())


# soap_descriptors = soap.create( atoms,centers=[0])

# Parse Lammps dump file

In [4]:
!rm -r dumpFile;mkdir dumpFile

#--- fetch parameters
path = confParser['input files']['input_path']
indx = confParser['input files']['fileIndex']
dumpFile = '%s/%s'%(path,confParser['input files']['dump file'].split()[int(indx)])
datFile = '%s/%s'%(path,confParser['input files']['diffusion file'].split()[int(indx)])
lib_path = confParser['input files']['lib_path']
outpt = 'dumpFile/dump.xyz'
outpt_headers = 'dumpFile/calcResults.txt'

#--- parse dump: call ovito
t0=time.time()
!ovitos $lib_path/OvitosCna.py $dumpFile $outpt 1 7 $outpt_headers
print('output dump file=%s s'%(time.time()-t0))


#--- parse dump files
print('parsing %s'%(outpt))
t0=time.time()
lmpData = lp.ReadDumpFile( '%s'%(outpt) ) 
lmpData.GetCords( ncount = 1) #sys.maxsize)
print('elapsed time=%s s'%(time.time()-t0))

print('time steps:',lmpData.coord_atoms_broken.keys())
display(lmpData.coord_atoms_broken[0].head())

#--- add timescales
lmpData.times = np.loadtxt(datFile)[:,0]

#--- parse headers
lmpData.headers = pd.DataFrame(np.loadtxt(outpt_headers),columns=["Barrier", "Energy", "Step", "Time"])

AnalysisType= 7
InputFile= ./training/allconf
num_frames= 1667
frame=0
frame=1
frame=2
frame=3
frame=4
frame=5
frame=6
frame=7
frame=8
frame=9
frame=10
frame=11
frame=12
frame=13
frame=14
frame=15
frame=16
frame=17
frame=18
frame=19
frame=20
frame=21
frame=22
frame=23
frame=24
frame=25
frame=26
frame=27
frame=28
frame=29
frame=30
frame=31
frame=32
frame=33
frame=34
frame=35
frame=36
frame=37
frame=38
frame=39
frame=40
frame=41
frame=42
frame=43
frame=44
frame=45
frame=46
frame=47
frame=48
frame=49
frame=50
frame=51
frame=52
frame=53
frame=54
frame=55
frame=56
frame=57
frame=58
frame=59
frame=60
frame=61
frame=62
frame=63
frame=64
frame=65
frame=66
frame=67
frame=68
frame=69
frame=70
frame=71
frame=72
frame=73
frame=74
frame=75
frame=76
frame=77
frame=78
frame=79
frame=80
frame=81
frame=82
frame=83
frame=84
frame=85
frame=86
frame=87
frame=88
frame=89
frame=90
frame=91
frame=92
frame=93
frame=94
frame=95
frame=96
frame=97
frame=98
frame=99
frame=100
frame=101
frame=102
frame=103
frame=1

frame=827
frame=828
frame=829
frame=830
frame=831
frame=832
frame=833
frame=834
frame=835
frame=836
frame=837
frame=838
frame=839
frame=840
frame=841
frame=842
frame=843
frame=844
frame=845
frame=846
frame=847
frame=848
frame=849
frame=850
frame=851
frame=852
frame=853
frame=854
frame=855
frame=856
frame=857
frame=858
frame=859
frame=860
frame=861
frame=862
frame=863
frame=864
frame=865
frame=866
frame=867
frame=868
frame=869
frame=870
frame=871
frame=872
frame=873
frame=874
frame=875
frame=876
frame=877
frame=878
frame=879
frame=880
frame=881
frame=882
frame=883
frame=884
frame=885
frame=886
frame=887
frame=888
frame=889
frame=890
frame=891
frame=892
frame=893
frame=894
frame=895
frame=896
frame=897
frame=898
frame=899
frame=900
frame=901
frame=902
frame=903
frame=904
frame=905
frame=906
frame=907
frame=908
frame=909
frame=910
frame=911
frame=912
frame=913
frame=914
frame=915
frame=916
frame=917
frame=918
frame=919
frame=920
frame=921
frame=922
frame=923
frame=924
frame=925
frame=926


frame=1611
frame=1612
frame=1613
frame=1614
frame=1615
frame=1616
frame=1617
frame=1618
frame=1619
frame=1620
frame=1621
frame=1622
frame=1623
frame=1624
frame=1625
frame=1626
frame=1627
frame=1628
frame=1629
frame=1630
frame=1631
frame=1632
frame=1633
frame=1634
frame=1635
frame=1636
frame=1637
frame=1638
frame=1639
frame=1640
frame=1641
frame=1642
frame=1643
frame=1644
frame=1645
frame=1646
frame=1647
frame=1648
frame=1649
frame=1650
frame=1651
frame=1652
frame=1653
frame=1654
frame=1655
frame=1656
frame=1657
frame=1658
frame=1659
frame=1660
frame=1661
frame=1662
frame=1663
frame=1664
frame=1665
frame=1666
output dump file=16.843087673187256 s
parsing dumpFile/dump.xyz
elapsed time=0.037982940673828125 s
time steps: dict_keys([0, 1])


,id,type,x,y,z
0,1,2,3.867879,18.055094,7.422952
1,2,1,9.194937,2.046315,3.873742
2,3,3,9.203422,21.626698,2.083052
3,4,3,2.095637,5.627621,7.422738
4,5,1,18.064135,12.749897,2.083203


# Energy

In [9]:
class EnergyBarrier:
    '''
    return energy barriers corresponding to diffusional hopping
    '''
    def __init__(self,
                 events_directory,evlist_directory,
                 lmpData,
                 nmax = sys.maxsize,
                 verbose=False):
        self.events_dir = events_directory
        self.evlist_dir = evlist_directory
        self.lmpData = lmpData.coord_atoms_broken[0]
        
        self.nmax = np.min(np.array([nmax,sys.maxsize]))
        self.verbose = verbose
        
    def Parse(self):
        '''
        parse event files
        '''
        self.diffusionPaths   = self.ParseDiffusionPath()
        self.events_id_energy = self.ParseEvents_dir()
        self.catalog          = self.ParseEvList_dir()
        
        
    def ParseEvents_dir(self):
        '''
        return energy barriers associated with hopping events
        '''
        files = os.listdir(self.events_dir)
        d=[]
        count = 0
        for sfile in files:
            if not '.xyz' in sfile: #--- skip .xyz files 
                try:
                    filee=open('%s/%s'%(self.events_dir,sfile)) #--- open file
                    xstrs = filee.readlines()
                    event_id = int(xstrs[0].split()[-1]) #--- event id
                    barrier = float(xstrs[2].split()[-1]) #--- energy
                    ncluster =  int(xstrs[15].split()[-1])                 
                    shape_cluster_atoms =  int(xstrs[16].split()[-1])
                    atom_id = int(xstrs[17+ncluster].split()[0])
                    if self.verbose:
                        print('file=%s atom_id=%s'%(sfile,atom_id))
                    d = np.c_[event_id,atom_id,barrier] if len(d) == 0 else\
                    np.concatenate([d,np.c_[event_id,atom_id,barrier]])
                    #
                    if count > self.nmax:
                        break
                    count += 1
                except:
#                    traceback.print_exc()
                    continue
            
        #--- extract types
        df       = self.lmpData
        atom_ids = d[:,1]
        types    = utl.FilterDataFrame(df, 
                    key='id', 
                    val=atom_ids
                   )['type']

        return pd.DataFrame(np.c_[types,d],columns=['atom_type','event_id','atom_id','barrier'])

    @staticmethod
    def ReadConfigurationFile(dumpFile, outpt, outpt_headers):
        '''
        return displacements
        '''
        !rm outpt outpt_headers
        lib_path = confParser['input files']['lib_path']

        #--- parse dump: call ovito
        !ovitos $lib_path/OvitosCna.py $dumpFile $outpt 1 7 $outpt_headers

        #--- parse dump files
        lmpData = lp.ReadDumpFile( '%s'%(outpt) ) 
        lmpData.GetCords( ncount = sys.maxsize)

        #--- disp
        fileCurr = fileRef = outpt #'dumpFile/dump.xyz'
        outpt = 'disp.xyz'
        !rm $outpt

        #--- call ovito
        os.system('ovitos %s/OvitosCna.py %s %s 1 8 %s'%(lib_path,fileCurr,outpt,fileRef))

        #--- parse disp files
        lmpDisp = lp.ReadDumpFile( outpt )
        lmpDisp.GetCords( ncount = sys.maxsize )
        return lmpDisp

    @staticmethod
    def GetEventid(filestr):
        '''
        extract event id from input string
        '''
        n = len(filestr)
        return int(filestr[5:n-4])
    
    def ParseDiffusionPath(self):
        '''
        return diffusion path corresponding to hopping events
        '''
        disps = {}
        count = 0
        files = os.listdir(self.events_dir)
        for sfile in files:
            if '.xyz' in sfile: #--- skip .xyz files 
                try:
                    dumpFile = '%s/%s'%(self.events_dir,sfile) #--- open file
                    #         outpt = 'dumpFile/dump.xyz'
                    event_id = EnergyBarrier.GetEventid(sfile)
                    disps[event_id] = \
                    EnergyBarrier.ReadConfigurationFile(dumpFile, 'dump.xyz', 'calcResults.txt')
                    if self.verbose:
                        print('file=%s atom_id=%s'%(sfile,atom_id))
                    #
                    if count > self.nmax:
                        break
                    count += 1
                except:
                    traceback.print_exc()
                    continue
        return disps
    
    def ParseEvList_dir(self):
        files = os.listdir(self.evlist_dir)
        events={}
        for sfile in files:
            try:
                kmc_step = int(sfile.split('_')[-1])
        #        print(kmc_step)
                filee=open('%s/%s'%(self.evlist_dir,sfile)) #--- open file
                events[kmc_step] = pd.read_csv(filee,delim_whitespace=True).iloc[1:]#delimiter='')
            except:
                continue
        return events
        
    def SplitByType(self):
        '''
        return energies (parsed from catalogs) slipt by atom types
        '''
        kmc_steps = list(self.catalog.keys())
        kmc_steps.sort()


        #--- dict based on types
        df_concat = {}
        types = list(set(self.lmpData.type))
        for itype in types:
            df_concat[str(itype)] = {}

        for kmc_step in kmc_steps: #--- kmc loop
            df = self.catalog[kmc_step]
            sdict=df.groupby(by='#TypeId').groups #--- group by type
            for itype in sdict:
                indices = sdict[itype] #--- row index: atoms with  '#TypeId' == itype
                cond = len(df_concat[itype]) == 0 #--- empty key?
                df_concat[itype] = np.c_[df.loc[indices]] if cond else\
                np.concatenate([df_concat[itype],np.c_[df.loc[indices]]],axis=0)

        self.energyByType = {}
        for itype in df_concat:
             self.energyByType[ itype ] = pd.DataFrame(df_concat[itype],columns=list(df.keys()))        





## main()

In [11]:
def main():
    if not eval(confParser['EnergyBarrier']['EnergyBarrier']):
        return
    
    eb = EnergyBarrier('%s/EVENTS_DIR'%confParser['input files']['input_path'],
                       '%s/EVLIST_DIR'%confParser['input files']['input_path'],
                       lmpData,
                       nmax = 10,
#                       verbose = True,

                      )
    eb.Parse()
    pdb.set_trace()
    
    udisp = eb.diffusionPaths[71729].coord_atoms_broken[2].['']
    positions = eb.diffusionPaths[71729].coord_atoms_broken[0].['']
    energies = ??
#    eb.SplitByType()

    #eb.events_id_energy extract from Events_dir
    #eb.energyByType extract from catalogs

    #--- write to file
    with open('eventID_barrier.txt','w') as fp:
        np.savetxt(fp,
                   np.c_[eb.events_id_energy],
                   header='atom_type event_id atom_id barrier')

    #--- write to file: energy from catalogs
#     for itype in eb.energyByType.keys():
#         with open('eventID_barrier_catalog_type%s.txt'%itype,'w') as fp:
#         #--- concat different types
#             sarr = np.c_[eb.energyByType[itype][['AtomId','eventId','barrier']]].astype(float)
#             np.savetxt(fp,
#                        sarr,
#                        header='AtomId eventId barrier'
#                       )
main()

71729
rm: outpt: No such file or directory
rm: outpt_headers: No such file or directory
AnalysisType= 7
InputFile= ./training/EVENTS_DIR/event71729.xyz
num_frames= 3
frame=0
frame=1
frame=2
reached end of file!
AnalysisType= 8
InputFile= dump.xyz
num_frames= 3
frame=0
frame=1
frame=2
reached end of file!
499972
rm: outpt: No such file or directory
rm: outpt_headers: No such file or directory
AnalysisType= 7
InputFile= ./training/EVENTS_DIR/event499972.xyz
num_frames= 3
frame=0
frame=1
frame=2
reached end of file!
AnalysisType= 8
InputFile= dump.xyz
num_frames= 3
frame=0
frame=1
frame=2
reached end of file!
505597
rm: outpt: No such file or directory
rm: outpt_headers: No such file or directory
AnalysisType= 7
InputFile= ./training/EVENTS_DIR/event505597.xyz
num_frames= 3
frame=0
frame=1
frame=2
reached end of file!
AnalysisType= 8
InputFile= dump.xyz
num_frames= 3
frame=0
frame=1
frame=2
reached end of file!
479138
rm: outpt: No such file or directory
rm: outpt_headers: No such file or

ipdb> eb.diffusionPaths[71729].coord_atoms_broken.keys()
dict_keys([0, 1, 2])
ipdb> eb.diffusionPaths[71729].coord_atoms_broken[0]
    id  type         x         y         z  DisplacementX  DisplacementY  \
0    1     2 -0.029191  0.019373  3.490535            0.0            0.0   
1    2     1  3.559232 -0.044688  0.027040            0.0            0.0   
2    3     3 -0.024697 -0.041222 -3.566228            0.0            0.0   
3    4     2 -3.633918 -0.016818 -0.061608            0.0            0.0   
4    5     1 -0.012350 -3.536597  0.019283            0.0            0.0   
5    6     2 -0.032119  3.493144 -0.028305            0.0            0.0   
6    7     2  0.057069 -1.747699  1.756996            0.0            0.0   
7    8     1 -0.029879  1.711989  1.784421            0.0            0.0   
8    9     1  1.737637 -1.761105  0.043655            0.0            0.0   
9   10     3  1.764953  1.733996  0.019511            0.0            0.0   
10  11     1 -1.822327 -0.024504 

# Train NN

In [ ]:
dir(lp)